In [1]:
import pandas as pd
cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("sample_data/training.1600000.processed.noemoticon.csv", names=cols,encoding = 'latin')
df.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:
df.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [3]:
df_pos_total = df[df['sentiment'] == 4]
df_neg_total = df[df['sentiment'] == 0]
df_pos_total['sentiment'] = 2
df_neg_total['sentiment'] = 0
df_pos = df_pos_total.iloc[:int(len(df_pos_total)/100)]
df_neg = df_neg_total.iloc[:int(len(df_neg_total)/100)]
df_new = pd.concat([df_pos, df_neg])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
#Data preprocessing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
lst= []
for line in df_new.text:
  tokens = word_tokenize(line)
  tokens = [w.lower() for w in tokens]    #to lower case
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]    #strip spaces 
  words = [word for word in stripped if word.isalpha()]    #emove numbers
  stop_words = set(stopwords.words('english'))      
  words = [w for w in words if not w in stop_words]     #remove stopwords
  lst.append(' '.join(words))
df_new.text = lst 

In [23]:
#convert text to numerical using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=3000)
vectors = vectorizer.fit_transform(df_new.text)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

,abit,able,absolutely,abt,access,accident,accidentally,account,accounting,accounts,ace,ache,aches,aching,across,acting,actual,actually,ad,add,added,addicted,addicting,addictive,adding,adobe,adorable,ads,adventure,advertising,advice,afford,afraid,afternoon,age,ages,ago,agree,agreed,ah,...,xxxx,ya,yah,yahoo,yall,yankees,yawn,yay,yayy,yayyy,yea,yeah,year,years,yeh,yellow,yep,yes,yest,yesterday,yet,yey,yo,yoga,yoghurt,york,young,youre,youtube,yr,yrs,yuck,yucky,yum,yummy,yup,zealand,zero,zombie,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.401586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
X = words_df
y = df_new.sentiment

In [25]:
from sklearn.naive_bayes import MultinomialNB

In [26]:
%%time
bayes = MultinomialNB()
bayes.fit(X, y)

CPU times: user 328 ms, sys: 13 ms, total: 341 ms
Wall time: 222 ms


In [29]:
df_new['prediction'] = bayes.predict(words_df)
df_new['probability'] = bayes.predict_proba(words_df)[:,1]

In [30]:
df_new['prediction']

800000    2
800001    0
800002    2
800003    0
800004    2
         ..
7995      0
7996      2
7997      0
7998      2
7999      0
Name: prediction, Length: 16000, dtype: int64

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [32]:
print("Training Naive Bayes")
bayes.fit(X_train, y_train)

Training Naive Bayes


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
from sklearn.metrics import confusion_matrix
y_true = y_test
y_pred = bayes.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted negative,Predicted positive
Is negative,1485,478
Is positive,581,1456


In [34]:
y_true = y_test
y_pred = bayes.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted negative,Predicted positive
Is negative,0.756495,0.243505
Is positive,0.285223,0.714777


In [35]:
df_new.loc[(df_new.probability >0.40) & (df_new.probability <0.60),'prediction' ] = 1

In [36]:
df_new.prediction.value_counts()

0    5579
2    5437
1    4984
Name: prediction, dtype: int64

In [37]:
df_new.to_csv("sample_data/trainingForBERT.csv", index= False)

In [39]:
import pickle
pickle.dump(bayes, open("mnb.sav", 'wb'))